<a href="https://colab.research.google.com/github/bkkaggle/pytorch-CycleGAN-and-pix2pix/blob/master/CycleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Take a look at the [repository](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix) for more information

# Install

In [ ]:
# !git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [1]:
import os
!ls .
# cd ./proj2/pytorch-CycleGAN-and-pix2pix
# os.chdir('pytorch-CycleGAN-and-pix2pix/')

CycleGAN.ipynb	data		 imgs		requirements.txt  util
LICENSE		datasets	 models		scripts
README.md	docs		 options	test.py
checkpoints	environment.yml  pix2pix.ipynb	train.py


In [2]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.


# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_cyclegan_dataset.sh [apple2orange, summer2winter_yosemite, horse2zebra, monet2photo, cezanne2photo, ukiyoe2photo, vangogh2photo, maps, cityscapes, facades, iphone2dslr_flower, ae_photos]`

Or use your own dataset by creating the appropriate folders and adding in the images.

-   Create a dataset folder under `/dataset` for your dataset.
-   Create subfolders `testA`, `testB`, `trainA`, and `trainB` under your dataset's folder. Place any images you want to transform from a to b (cat2dog) in the `testA` folder, images you want to transform from b to a (dog2cat) in the `testB` folder, and do the same for the `trainA` and `trainB` folders.

In [ ]:
!bash ./datasets/download_cyclegan_dataset.sh horse2zebra

In [2]:
# unzip dataset
import zipfile
import os
import os.path

path = r'/scratch/jz4364/dog2cat_afhq'
dog = r'/scratch/jz4364/dog2cat_afhq/testB.zip'
with zipfile.ZipFile(dog,'r') as zipf:
    zipf.extractall(path)

# Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_cyclegan_model.sh [apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [3]:
!bash ./scripts/download_cyclegan_model.sh apple2orange

Note: available models are apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower
Specified [apple2orange]
for details.

--2022-04-20 12:29:40--  http://efrosgans.eecs.berkeley.edu/cyclegan/pretrained_models/apple2orange.pth
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.244.190
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.244.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45575747 (43M)
Saving to: ‘./checkpoints/apple2orange_pretrained/latest_net_G.pth’

./checkpoints/apple 100%[===================>]  43.46M  29.2MB/s    in 1.5s    

2022-04-20 12:29:42 (29.2 MB/s) - ‘./checkpoints/apple2orange_pretrained/latest_net_G.pth’ saved [45575747/45575747]



# Training

-   `python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. I've found that a batch size of 16 fits onto 4 V100s and can finish training an epoch in ~90s.

Once your model has trained, copy over the last checkpoint to a format that the testing model can automatically detect:

Use `cp ./checkpoints/horse2zebra/latest_net_G_A.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class A to class B and `cp ./checkpoints/horse2zebra/latest_net_G_B.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class B to class A.


In [10]:
import os
!ls
file_list = os.listdir(r"./datasets/dog2cat/trainA")
print(len(file_list))
file_list = os.listdir(r"./datasets/dog2cat/trainB")
print(len(file_list))

CycleGAN.ipynb	data		 imgs		requirements.txt  train.py
LICENSE		datasets	 models		results		  util
README.md	docs		 options	scripts		  wandb
checkpoints	environment.yml  pix2pix.ipynb	test.py
11352
7913


In [ ]:
!python train.py --dataroot /scratch/jz4364/dog2cat --name dog2cat --model cycle_gan --use_wandb

In [ ]:
# using afhq dataset
!python train.py --dataroot /scratch/jz4364/dog2cat_afhq --name dog2cat_afhq --model cycle_gan --use_wandb --lambda_A
 5 --lambda_B 5

# Testing

-   `python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout`

Change the `--dataroot` and `--name` to be consistent with your trained model's configuration.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> The option --model test is used for generating results of CycleGAN only for one side. This option will automatically set --dataset_mode single, which only loads the images from one set. On the contrary, using --model cycle_gan requires loading and generating results in both directions, which is sometimes unnecessary. The results will be saved at ./results/. Use --results_dir {directory_path_to_save_result} to specify the results directory.

> For your own experiments, you might want to specify --netG, --norm, --no_dropout to match the generator architecture of the trained model.

In [ ]:
!python test.py --dataroot datasets/apple2orange/testA --name apple2orange_pretrained --model test --no_dropout --use_wandb

In [ ]:
!python test.py --dataroot /scratch/jz4364/dog2cat_afhq --name dog2cat_afhq --model cycle_gan --no_dropout --use_wandb

# Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_fake.png')
plt.imshow(img)

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_real.png')
plt.imshow(img)

In [1]:
import wandb
api = wandb.Api()
run = api.run("/klchg/CycleGAN-and-pix2pix/runs/1u6afnzt")
print(run.history())

     _step                                             real_A  \
0        3  {'_type': 'image-file', 'format': 'png', 'heig...   
1       10                                                NaN   
2       20  {'_type': 'image-file', 'format': 'png', 'heig...   
3       21                                                NaN   
4       25  {'height': 256, 'path': 'media/images/real_A_2...   
..     ...                                                ...   
479   2655  {'width': 256, '_type': 'image-file', 'format'...   
480   2672                                                NaN   
481   2674                                                NaN   
482   2685                                                NaN   
483   2691                                                NaN   

                                                real_B  \
0    {'_type': 'image-file', 'format': 'png', 'heig...   
1                                                  NaN   
2    {'size': 129796, 'width': 256, '_type': 

In [2]:
wandb.init(project="CycleGAN-and-pix2pix", entity="dl_proj2")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: klchg (use `wandb login --relogin` to force relogin)


In [1]:
import wandb
api = wandb.Api()
run = api.run("/klchg/CycleGAN-and-pix2pix/runs/1u6afnzt")
run.summary['Result']

{'sha256': 'ed65c922e9b173a2c53c8dda8cbbb42ce83936056607a2f7382d0b96fa564b7b', 'artifact_path': 'wandb-client-artifact://15vhjoy06m14enrkzhuql1q75xv898sfit887lfrfbg6r26zsfc0p32ox0kgrflse9z64kf0kahd5ye44r9r6puvbjcgvwk5r96r9wedobch1j37uc4lxxb0j512bjwn:latest/Result.table.json', '_latest_artifact_path': 'wandb-client-artifact://15vhjoy06m14enrkzhuql1q75xv898sfit887lfrfbg6r26zsfc0p32ox0kgrflse9z64kf0kahd5ye44r9r6puvbjcgvwk5r96r9wedobch1j37uc4lxxb0j512bjwn:latest/Result.table.json', 'path': 'media/table/Result_2688_ed65c922e9b173a2c53c.table.json', 'size': 187, '_type': 'table-file', 'ncols': 9, 'nrows': 1}